# Training Notebook

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ml_attack import check_secret, clean_secret, get_no_mod, LWEDataset, get_filename_from_params, get_default_params, get_vector_distribution

from ml_attack.train import LinearComplex, train_until_stall

import numpy as np
import torch
import torch.nn as nn

from sklearn.metrics import confusion_matrix, classification_report

## Dataset creation

Training debug:

In [2]:
LWEDataset.params_from_file("./../data/data_n_128_k_1_s_binary_4d1b9.pkl")

{'n': 128,
 'q': 3329,
 'k': 1,
 'secret_type': 'binary',
 'eta': 2,
 'gaussian_std': 2,
 'hw': -1,
 'error_type': 'cbd',
 'num_gen': 1,
 'add_noise': True,
 'mod_q': True,
 'seed': 2,
 'float_type': 'double',
 'reduction_std': 2,
 'reduction_factor': 0.875,
 'reduction_resampling': True,
 'min_samples': 0,
 'approximation_std': 3,
 'algos': ['flatter', 'BKZ2.0'],
 'lookback': 3,
 'bkz_block_sizes': [30, 40],
 'bkz_deltas': [0.96, 0.99],
 'flatter_alphas': [0.04, 0.025],
 'penalty': 4,
 'verbose': True,
 'checkpoint_filename': 'checkpoints/checkpoint_6186895',
 'reload_checkpoint': False,
 'save_to': '../data'}

In [3]:
params = get_default_params()
params.update({
    'n': 100,
    'secret_type': 'binary',

    'num_gen': 1,
    'seed': 2,

    'reduction_std': 2,
    'reduction_factor': 0.875,
    'reduction_resampling': True,
    'approximation_std': 2,
    
    'penalty': 4,
    'verbose': True
})

filename = get_filename_from_params(params, dir='./../data')

filename = "./../data/data_n_100_k_1_s_cbd_1adc8.pkl"

reload = True
if os.path.exists(filename) and reload:
    print(f"Loading dataset from {filename}")
    dataset = LWEDataset.from_file(filename)
    params = dataset.params
else:
    print(f"Generating dataset and saving to {filename}")
    dataset = LWEDataset(params)
    dataset.initialize()
    dataset.reduce()
    dataset.approximate_b()
    dataset.save(filename)

Loading dataset from ./../data/data_n_100_k_1_s_cbd_1adc8.pkl


In [4]:
in_candidates = 0
exact_candidates = 0
indeces = []

dataset.params["approximation_std"] = 3
dataset.approximate_b()

secret = dataset.get_secret()
A_reduced = dataset.get_A()
b_reduced = dataset.get_B()

b_real = get_no_mod(params, A_reduced, secret, b_reduced)

mask = np.zeros(A_reduced.shape[0], dtype=np.bool)
c = np.zeros(A_reduced.shape[0], dtype=np.int32)

for i in range(len(dataset)):
    true_b = b_real[i]

    candidates = dataset.b_candidates[i]
    probs = dataset.b_probs[i]
    c[i] = (true_b - candidates[np.argmax(probs)]) // params['q']

    if true_b in candidates:
        in_candidates += 1
        if true_b == candidates[np.argmax(probs)]:
            exact_candidates += 1
            mask[i] = True
        else:
            #print(f"Index {i}: {true_b} not the best candidate: {candidates[np.argmax(probs)]} with prob {np.max(probs)}")
            #print(f"Other candidates: {candidates} with probs {probs}")
            indeces.append(i)

    else:
        #print(f"Index {i}: {true_b} not in set: {candidates}")
        indeces.append(i)
    
print(f"True B in candidate set: {in_candidates} / {len(dataset)} ({100 * in_candidates / len(dataset):.2f}%)")
print(f"True B is the best candidate: {exact_candidates} / {len(dataset)} ({100 * exact_candidates / len(dataset):.2f}%)")
print(f"Indeces: {indeces}")
print(f"C values: {set(c.tolist())}")

max_probs = [max(probs) for probs in dataset.b_probs]
expected_success_rate = np.mean(max_probs)
print(f"Expected success rate: {expected_success_rate * 100:.2f}%")


True B in candidate set: 933 / 935 (99.79%)
True B is the best candidate: 677 / 935 (72.41%)
Indeces: [2, 4, 6, 9, 13, 15, 22, 25, 32, 37, 38, 40, 41, 44, 45, 46, 48, 51, 53, 54, 56, 57, 58, 60, 63, 68, 71, 72, 78, 80, 84, 88, 96, 102, 103, 109, 112, 113, 114, 115, 122, 123, 124, 129, 142, 144, 148, 153, 155, 158, 160, 161, 164, 165, 167, 168, 170, 172, 177, 178, 183, 190, 191, 192, 194, 198, 200, 201, 204, 209, 210, 211, 212, 213, 218, 219, 220, 222, 223, 227, 232, 238, 242, 248, 257, 258, 261, 263, 266, 276, 277, 279, 281, 287, 288, 290, 292, 294, 297, 303, 304, 313, 314, 315, 324, 329, 333, 339, 340, 348, 353, 359, 363, 366, 370, 371, 372, 377, 378, 407, 410, 414, 416, 422, 425, 429, 432, 436, 437, 438, 444, 451, 457, 462, 466, 467, 472, 477, 478, 480, 484, 490, 496, 505, 513, 517, 519, 525, 535, 544, 552, 568, 574, 580, 583, 586, 587, 588, 591, 592, 596, 597, 598, 599, 601, 602, 603, 605, 606, 608, 613, 618, 623, 627, 630, 631, 633, 637, 638, 640, 641, 643, 645, 648, 650, 654, 657,

In [5]:
for idx, value in enumerate(b_real):
  print(f"Index {idx}: True B = {value}, Candidates = {dataset.b_candidates[idx]}, Probabilities = {dataset.b_probs[idx]}")

Index 0: True B = 781.0, Candidates = [-2548.   781.], Probabilities = [0.09464231 0.90535769]
Index 1: True B = 1420.0, Candidates = [-1909.  1420.], Probabilities = [0.35409847 0.64590153]
Index 2: True B = 1673.0, Candidates = [-1656.  1673.], Probabilities = [0.50399862 0.49600138]
Index 3: True B = 1269.0, Candidates = [-2060.  1269.], Probabilities = [0.34140698 0.65859302]
Index 4: True B = 1748.0, Candidates = [-1581.  1748.], Probabilities = [0.53327426 0.46672574]
Index 5: True B = 635.0, Candidates = [-2694.   635.  3964.], Probabilities = [0.15976811 0.81895481 0.02127708]
Index 6: True B = 1918.0, Candidates = [-1411.  1918.], Probabilities = [0.61852431 0.38147569]
Index 7: True B = -808.0, Candidates = [-808. 2521.], Probabilities = [0.84568112 0.15431888]
Index 8: True B = -81.0, Candidates = [-3410.   -81.  3248.], Probabilities = [0.04061625 0.90520708 0.05417667]
Index 9: True B = -1884.0, Candidates = [-1884.  1445.], Probabilities = [0.38677565 0.61322435]
Index 10

Index 362: True B = -838.0, Candidates = [-4167.  -838.  2491.], Probabilities = [0.02197396 0.745285   0.23274104]
Index 363: True B = -1757.0, Candidates = [-1757.  1572.], Probabilities = [0.46900978 0.53099022]
Index 364: True B = -480.0, Candidates = [-3809.  -480.  2849.], Probabilities = [0.04633806 0.7888361  0.16482584]
Index 365: True B = 1581.0, Candidates = [-1748.  1581.], Probabilities = [0.47334935 0.52665065]
Index 366: True B = 2009.0, Candidates = [-4649. -1320.  2009.], Probabilities = [0.01024293 0.60946579 0.38029128]
Index 367: True B = 1592.0, Candidates = [-5066. -1737.  1592.  4921.], Probabilities = [0.0148505  0.46558183 0.5010573  0.01851037]
Index 368: True B = -1506.0, Candidates = [-4835. -1506.  1823.], Probabilities = [0.01209567 0.54073053 0.4471738 ]
Index 369: True B = -636.0, Candidates = [-3965.  -636.  2693.], Probabilities = [0.01981416 0.82449083 0.15569501]
Index 370: True B = -3250.0, Candidates = [-3250.    79.  3408.], Probabilities = [0.101

In [6]:
A_reduced = dataset.get_A()
best_b = dataset.best_b_candidates

# Check if GPU is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LinearComplex(params).to(device)
A_reduced = torch.tensor(A_reduced, dtype=torch.float).to(device)
best_b = torch.tensor(best_b, dtype=torch.float).to(device)

epoch = 0
loss, epoch = train_until_stall(model, A_reduced, best_b, dataset, epoch=epoch)
if loss == 0:
    print("Secret guessed correctly at epoch {}!".format(epoch))
else:
    print(f"Stalling detected at loss {loss:.4f}.")

Epoch 0, Loss: 1458.6511
Epoch 1, Loss: 1457.6808
Epoch 2, Loss: 1456.7109
Epoch 3, Loss: 1455.7444
Epoch 4, Loss: 1454.7783
Epoch 5, Loss: 1453.8126
Epoch 6, Loss: 1452.8467
Epoch 7, Loss: 1451.8811
Epoch 8, Loss: 1450.9163
Epoch 9, Loss: 1449.9519
Epoch 10, Loss: 1448.9879
Epoch 11, Loss: 1448.0238
Epoch 12, Loss: 1447.0599
Epoch 13, Loss: 1446.0968
Epoch 14, Loss: 1445.1364
Epoch 15, Loss: 1444.1763
Epoch 16, Loss: 1443.2166
Epoch 17, Loss: 1442.2568
Epoch 18, Loss: 1441.2994
Epoch 19, Loss: 1440.3455
Epoch 20, Loss: 1439.3917
Epoch 21, Loss: 1438.4385
Epoch 22, Loss: 1437.4865
Epoch 23, Loss: 1436.5348
Epoch 24, Loss: 1435.5829
Epoch 25, Loss: 1434.6311
Epoch 26, Loss: 1433.6793
Epoch 27, Loss: 1432.7275
Epoch 28, Loss: 1431.7760
Epoch 29, Loss: 1430.8280
Epoch 30, Loss: 1429.8805
Epoch 31, Loss: 1428.9333
Epoch 32, Loss: 1427.9861
Epoch 33, Loss: 1427.0391
Epoch 34, Loss: 1426.0934
Epoch 35, Loss: 1425.1516
Epoch 36, Loss: 1424.2107
Epoch 37, Loss: 1423.2703
Epoch 38, Loss: 1422.3

In [7]:
from sklearn.mixture import BayesianGaussianMixture
import numpy as np

A_numpy = A_reduced.cpu().numpy()
b_numpy = best_b.cpu().numpy()

# Reshape for EM (treat y as part of the data)
data = np.column_stack([A_numpy, b_numpy])

# EM clustering (BayesianGaussianMixture avoids needing to specify K)
em = BayesianGaussianMixture(n_components=5, max_iter=10000)
clusters = em.fit_predict(data)

# Recover s_k for each cluster
for k in np.unique(clusters):
    X_k = A_numpy[clusters == k]
    y_k = b_numpy[clusters == k]

    epoch = 0
    model = LinearComplex(params).to(device)
    loss, epoch = train_until_stall(model, torch.tensor(X_k, dtype=torch.float).to(device), torch.tensor(y_k, dtype=torch.float).to(device), dataset, epoch=epoch, verbose=False)
    
    # Check percentage of real b in y_k for this cluster
    real_b_in_y_k = np.isin(b_real[clusters == k], y_k)
    percentage = 100 * np.sum(real_b_in_y_k) / len(y_k)
    print(f"Cluster {k}: {percentage:.2f}% of real b in y_k")
    print(f"Cluster {k}: Loss = {loss:.4f}, Epoch = {epoch}")

Cluster 0: 81.11% of real b in y_k
Cluster 0: Loss = 258.0245, Epoch = 4595
Cluster 1: 78.89% of real b in y_k
Cluster 1: Loss = 387.1187, Epoch = 3503
Cluster 2: 82.23% of real b in y_k
Cluster 2: Loss = 95.2910, Epoch = 3163
Cluster 3: 55.62% of real b in y_k
Cluster 3: Loss = 496.5911, Epoch = 5830
Cluster 4: 61.81% of real b in y_k
Cluster 4: Loss = 590.1509, Epoch = 4471


In [8]:
from sklearn.linear_model import RANSACRegressor

def iterative_ransac(X, y, max_clusters=5, eps=0.5):
    remaining_X, remaining_y = X.copy(), y.copy()
    clusters = -np.ones(len(y))
    remaining_indices = np.arange(len(y))
    for k in range(max_clusters):
        if len(remaining_X) == 0:
            break
        ransac = RANSACRegressor(residual_threshold=eps)
        ransac.fit(remaining_X, remaining_y)
        inliers = ransac.inlier_mask_

        # Assign cluster labels to the original indices
        clusters[remaining_indices[inliers]] = k

        # Check how many inliers correspond to the true b values
        real_b_in_inliers = np.isin(remaining_y[inliers], b_real)
        num_real_b_in_inliers = np.sum(real_b_in_inliers)
        
        # Calculate and print the percentage of inliers that are real b values
        percentage_real_b = 100 * num_real_b_in_inliers / np.sum(inliers) if np.sum(inliers) > 0 else 0.0
        print(f"Cluster {k}: {percentage_real_b:.2f}% of inliers are real_b")

        # Remove inliers for next iteration
        remaining_X = remaining_X[~inliers]
        remaining_y = remaining_y[~inliers]
        remaining_indices = remaining_indices[~inliers]
    return clusters

clusters = iterative_ransac(A_numpy, b_numpy, max_clusters=5, eps=0.5)

Cluster 0: 78.43% of inliers are real_b
Cluster 1: 79.41% of inliers are real_b
Cluster 2: 80.39% of inliers are real_b
Cluster 3: 79.41% of inliers are real_b
Cluster 4: 80.39% of inliers are real_b


In [9]:
from sklearn.linear_model import RANSACRegressor, HuberRegressor

# Flatten all candidates and keep track of their original indices
all_candidates = []
all_x = []
weights = []  # optional: based on probabilities

for i in range(len(A_numpy)):
    for j, (cand, prob) in enumerate(zip(dataset.b_candidates[i], dataset.b_probs[i])):
        all_candidates.append(cand)
        all_x.append(A_numpy[i])
        weights.append(prob)  # optional: weight by probability

all_x = np.array(all_x)
all_candidates = np.array(all_candidates)
weights = np.array(weights)  # optional
sigma = params['q']

# Fit RANSAC (use Huber regressor for robustness to noise)
ransac = RANSACRegressor(
    estimator=HuberRegressor(max_iter=10000),
    residual_threshold=3 * sigma,  # adjust based on noise level
    min_samples= int(len(A_numpy) * 0.9),
    max_trials=10000,
)
ransac.fit(all_x, all_candidates, sample_weight=weights)  # optional: weight by probabilities

# Get inlier mask (True for candidates that fit the model)
inlier_mask = ransac.inlier_mask_
inlier_candidates = all_candidates[inlier_mask]

percentage_inliers = 100 * np.sum(inlier_mask) / len(inlier_mask)
print(f"Percentage of inliers: {percentage_inliers:.2f}%")

# For each original b_i, select the candidate closest to the RANSAC prediction
predicted_b = ransac.predict(A_numpy)
recovered_b = []

for i in range(len(A_numpy)):
    possible_candidates = dataset.b_candidates[i]
    best_candidate = min(possible_candidates, key=lambda cand: abs(cand - predicted_b[i]))
    recovered_b.append(best_candidate)

# Calculate the percentage of recovered b values that match the true b values
correct_recovered = np.sum(np.array(recovered_b) == b_real)
percentage_correct = 100 * correct_recovered / len(b_real)
print(f"Percentage of recovered b values that match true b values: {percentage_correct:.2f}%")

Percentage of inliers: 100.00%
Percentage of recovered b values that match true b values: 72.51%


In [10]:
# Check the guessed secret
raw_guessed_secret = model.guessed_secret.cpu().detach().numpy()
guessed_secret = clean_secret(raw_guessed_secret, params)

real_secret = dataset.get_secret()

print("Raw Guessed secret:", raw_guessed_secret)
print("Guessed secret:", guessed_secret)
print("Actual secret:", real_secret)

Raw Guessed secret: [-0.73822224 -0.9078339   1.6613994  -1.9269019  -1.7003905   1.0524613
  0.20946646  0.14618878  0.37722954 -1.0586578   1.9159956   1.275429
  2.9518948  -1.0969965   0.24372263 -0.32958138 -1.427846    0.29590046
 -1.4407063   1.0972059   1.2134887  -0.61240315  0.09150084 -1.353555
 -0.17669794 -3.3762572  -2.3784742   0.07778508 -2.1999714  -0.15874991
 -1.4752699  -0.26311976  0.04170619 -1.1440461   1.3272914  -0.4871217
 -0.7052532  -0.57668823 -0.75785786 -2.9294975   1.243707    0.709606
  1.92655     0.27890566 -0.8017793   0.25464782  0.01884142  1.3795528
 -0.23828971 -0.59373456  1.5133961  -0.4683565  -0.20861414  0.8078171
  0.1254836  -1.3829086   0.9464974  -0.721527    1.4557453  -1.0913521
 -1.0529305   1.3367426  -0.21653563  1.083385   -0.01127556 -0.312452
  0.27767807 -0.13680454 -0.4251076  -0.23763019  0.45941323  0.3989796
  1.2505202   0.4332771  -0.9471539  -0.6332404  -1.9167536   1.3333071
 -1.0616493   0.50044084  3.3828545   1.441401

In [11]:
# Check the differences between the guessed and actual secret
diff = guessed_secret - real_secret
raw_diff = raw_guessed_secret[diff != 0]
raw_diff[raw_diff > params['q'] // 2] -= params['q']
diff_indices = np.nonzero(diff)
if len(diff[diff != 0]) > 0:
    print("Number of differences:", len(diff[diff != 0]))
    print("Difference:", raw_diff)
    print("real_secret:", real_secret[diff != 0])
    print("guessed_secret:", guessed_secret[diff != 0])
    print("Indices of differences:", diff_indices)

Number of differences: 68
Difference: [-0.73822224 -0.9078339   1.6613994  -1.9269019  -1.7003905   1.0524613
  0.20946646  0.14618878  0.37722954  1.9159956   1.275429    2.9518948
 -1.0969965   0.24372263 -1.4407063  -1.353555   -3.3762572  -2.3784742
  0.07778508 -2.1999714  -1.4752699  -0.26311976 -0.7052532  -0.57668823
 -0.75785786 -2.9294975   1.243707    0.709606    0.27890566  0.01884142
  1.5133961  -0.4683565  -0.20861414  0.8078171  -1.3829086  -0.721527
 -1.0913521  -1.0529305   1.3367426  -0.21653563  1.083385   -0.01127556
 -0.4251076  -0.23763019  1.2505202   0.4332771  -0.6332404  -1.9167536
  1.3333071  -1.0616493   3.3828545   1.4414014   2.4214559  -2.2183714
 -0.0702518  -0.22192456 -0.59298193 -1.9928168   0.87102395 -0.30201533
 -0.90982145 -0.7199186  -0.8187609   1.3281045   1.9180956   2.1664279
 -1.130398   -2.5755105 ]
real_secret: [ 1  2 -2  1 -1  0  1 -2  1 -1  2 -1  0  1  0  1  1  1 -1  0  1 -1  0  2
  1  0  0  0 -1 -1 -2 -1  1 -1  0  0  1  0 -1  1 -2 -1 

In [12]:
close_to_integer = np.abs(raw_guessed_secret - np.round(raw_guessed_secret))
sorted_indices = np.argsort(-close_to_integer)
print("Sorted uncertain indices:", sorted_indices)
print("Sorted uncertain values:", np.round(close_to_integer[sorted_indices], 3))

if len(diff_indices[0]) > 0:
  diff_indices_in_sorted = [np.where(sorted_indices == i)[0][0] for i in diff_indices[0]]
  print("Worst case scenario:", max(diff_indices_in_sorted))

Sorted uncertain indices: [79 35 50 30 51 70 98 58 81 18 73 16 68 97 37 82 86 49 71 21 55 80 47 26
  8 25 75 23  2 61 77 15 93 34 65 89  4 17 36 41 91 43 57 66 11 31  0 45
 72 14 40 38 48 69 85 83 62 20  6 52 28 44 53 92 24 95 29  7 33 67 96 88
 54 19 13  1 22 59 90 10 63 76 94 27 42  3 39 84 78  9 56 60 74  5 12 32
 46 64 99 87]
Sorted uncertain values: [0.5   0.487 0.487 0.475 0.468 0.459 0.456 0.456 0.441 0.441 0.433 0.428
 0.425 0.424 0.423 0.421 0.407 0.406 0.399 0.388 0.383 0.383 0.38  0.378
 0.377 0.376 0.367 0.354 0.339 0.337 0.333 0.33  0.328 0.327 0.312 0.302
 0.3   0.296 0.295 0.29  0.28  0.279 0.278 0.278 0.275 0.263 0.262 0.255
 0.251 0.244 0.244 0.242 0.238 0.238 0.222 0.218 0.217 0.213 0.209 0.209
 0.2   0.198 0.192 0.181 0.177 0.166 0.159 0.146 0.144 0.137 0.13  0.129
 0.125 0.097 0.097 0.092 0.092 0.091 0.09  0.084 0.083 0.083 0.082 0.078
 0.073 0.073 0.071 0.07  0.062 0.059 0.054 0.053 0.053 0.052 0.048 0.042
 0.019 0.011 0.009 0.007]
Worst case scenario: 99


In [13]:
from itertools import product

# Find values in raw_guessed_secret that are within ±0.1 of an integer
close_to_integer = np.abs(raw_guessed_secret - np.round(raw_guessed_secret)) < 0.2
uncertain_count = np.sum(~close_to_integer)
print("Number of uncertain values:", uncertain_count)

# Calculate the number of brute force attacks to perform
brute_force_attempts = 2 ** uncertain_count
print("Number of brute force attempts required:", brute_force_attempts)

# Get the indices of uncertain values
uncertain_indices = np.where(~close_to_integer)[0]

real_uncertain_secret = real_secret[uncertain_indices]
print("Real uncertain secret:", real_uncertain_secret)

# Perform brute force attack
raw_uncertain_secret = raw_guessed_secret[uncertain_indices]
raw_uncertain_secret[raw_uncertain_secret > params['q'] // 2] -= params['q']
raw_uncertain_secret = raw_uncertain_secret[np.abs(raw_uncertain_secret) <= params['eta']]

lower_values = np.floor(raw_uncertain_secret)
upper_values = np.ceil(raw_uncertain_secret)

#values = product(*zip(lower_values, upper_values))

#for value in values:
#    print("Trying values:", value)
    # Create a copy of the guessed secret
#    brute_force_secret = copy.deepcopy(guessed_secret)
    # Update the uncertain values with the current combination
#    for idx, val in zip(uncertain_indices, value):
#        brute_force_secret[idx] = val
    # Check if the guessed secret is correct
#    if check_secret(brute_force_secret, dataset.A, dataset.B, params):
#        print("Brute force attack successful! Guessed secret:", brute_force_secret)
#        break

Number of uncertain values: 60
Number of brute force attempts required: 1152921504606846976
Real uncertain secret: [ 1 -2 -1  1  1  2  1  0 -1  0  0  1 -1  1  1  1  1 -1  1  0  0  2  1  0
  0 -1  0  1  0 -1 -2 -1  1  0  0  1 -1  1  0  0  1 -1  0  0  0  1  1  0
  1  0  2  0 -1 -1  1  1  0  0  1  0]


In [14]:
def report(real_secret, guessed_secret):
    """
    Print classification report and confusion matrix.
    """
  
    # Get unique sorted labels and compute confusion matrix
    labels = np.unique(np.concatenate((real_secret, guessed_secret)))
    cm = confusion_matrix(real_secret, guessed_secret, labels=labels)

    # Header
    header = "       |" + "".join([f"{l:>6}" for l in labels]) + " | Accuracy"
    print("Confusion Matrix:")
    print(header)
    print("-" * len(header))

    # Rows
    for i, row in enumerate(cm):
        label = f"{labels[i]:>6} |"
        values = "".join([f"{v:6}" for v in row])

        correct = row[i]
        total = row.sum()
        acc = correct / total if total > 0 else 0.0
        print(label + values + f" | {acc:4.1%}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(real_secret, guessed_secret, zero_division=0))

report(real_secret, guessed_secret)

Confusion Matrix:
       |  -2.0  -1.0   0.0   1.0   2.0 | Accuracy
-------------------------------------------------
  -2.0 |     0     0     1     1     2 | 0.0%
  -1.0 |     3     7     9     3     3 | 28.0%
   0.0 |     3     8    17     6     2 | 47.2%
   1.0 |     4     9     8     7     1 | 24.1%
   2.0 |     0     3     0     2     1 | 16.7%

Classification Report:
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00         4
          -1       0.26      0.28      0.27        25
           0       0.49      0.47      0.48        36
           1       0.37      0.24      0.29        29
           2       0.11      0.17      0.13         6

    accuracy                           0.32       100
   macro avg       0.24      0.23      0.23       100
weighted avg       0.35      0.32      0.33       100

